In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import product

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=2)

<IB connected to 127.0.0.1:4004 clientId=2>

In [2]:
# Get the options list

options_url1 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=1"
options_url2 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=2"
nse = pd.concat([pd.read_html(options_url1, header=0)[2], 
                         pd.read_html(options_url2, header=0)[2]], 
                        sort=True).reset_index(drop=True)

nse = nse.iloc[:, [1, 2, 3]]
nse.columns = ['iSymbol', 'Desc', 'Symbol']

In [3]:
# Note multipliers are not correct for NSE!. So get them !!!
# Get the list of Equity and Index Option scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Replace & for M&M, L&T, etc. This is not needed for IBKR
mask = paisa.loc[paisa.Symbol.str.contains("&"), "Symbol"]
paisa.loc[paisa.Symbol.isin(mask), "Symbol"] = mask.str.replace("&", "")

nse_options = pd.merge(nse, paisa[['Symbol', 'Mlot']], on="Symbol", how='left')
nse_options["Exchange"] = 'NSE'

In [22]:
# List of nse indexes (extracted from error!)
idx_list = ["BANKNIFTY", "NIFTY50", "NSEFTSE", "USDINR"]
stock_list = nse_options.loc[~nse_options.iSymbol.isin(idx_list), :]

In [23]:
# Get the Stocks qualified
stocks = [Stock(symbol=x, exchange='NSE') for x in stock_list.iSymbol]
qual_stock = ib.qualifyContracts(*stocks)

[Stock(conId=44652144, symbol='ACC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ACC', tradingClass='ACC'),
 Stock(conId=56986798, symbol='ADANIENT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIENT', tradingClass='ADANIENT'),
 Stock(conId=79403918, symbol='ADANIPORT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ADANIPORTS', tradingClass='ADANIPORT'),
 Stock(conId=113744943, symbol='AMARAJABA', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='AMARAJABAT', tradingClass='AMARAJABA'),
 Stock(conId=44652129, symbol='AMBUJACEM', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='AMBUJACEM', tradingClass='AMBUJACEM'),
 Stock(conId=78607402, symbol='APOLLOHOS', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='APOLLOHOSP', tradingClass='APOLLOHOS'),
 Stock(conId=56988336, symbol='APOLLOTYR', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='APOLLO

In [33]:
# Get ALL expirations and strikes for the Stocks. Takes about 10 mins!!!
contracts = [(i.symbol, '', i.secType, i.conId) for i in qual_stock]
contracts_p = [ib.reqSecDefOptParams(*p) for p in contracts]

In [34]:
contracts_p

[[OptionChain(exchange='NSE', underlyingConId=44652144, tradingClass='ACC', multiplier='1', expirations={'20180927', '20180830', '20181025'}, strikes={1280.0, 1920.0, 900.0, 1540.0, 1160.0, 1800.0, 1420.0, 1040.0, 1680.0, 1300.0, 1940.0, 920.0, 1560.0, 1180.0, 1820.0, 1440.0, 1660.0, 1060.0, 1700.0, 1320.0, 940.0, 1580.0, 1200.0, 1840.0, 1460.0, 1080.0, 1720.0, 1340.0, 960.0, 1600.0, 1220.0, 1860.0, 1480.0, 1100.0, 1740.0, 1360.0, 980.0, 1620.0, 1240.0, 1880.0, 860.0, 1500.0, 1120.0, 1760.0, 1380.0, 1000.0, 1640.0, 1260.0, 1900.0, 880.0, 1520.0, 1140.0, 1780.0, 1400.0, 1020.0})],
 [OptionChain(exchange='NSE', underlyingConId=56986798, tradingClass='ADANIENT', multiplier='1', expirations={'20180927', '20180830', '20181025'}, strikes={130.0, 260.0, 135.0, 140.0, 145.0, 20.0, 150.0, 155.0, 30.0, 160.0, 165.0, 40.0, 170.0, 45.0, 175.0, 50.0, 180.0, 55.0, 185.0, 60.0, 190.0, 65.0, 195.0, 70.0, 200.0, 75.0, 205.0, 80.0, 210.0, 85.0, 215.0, 90.0, 220.0, 95.0, 225.0, 100.0, 230.0, 105.0, 235.0

In [63]:
# Filter symbol, Id, expiration and strike from chains and make a cartesian
chains = [(i.tradingClass, i.underlyingConId, i.expirations, i.strikes) for elem in contracts_p for i in elem]
L = [[[x[0]], [x[1]], sorted(x[2]), sorted(x[3])] for x in chains]

stock_chains = pd.DataFrame([j for i in L for j in product(*i)], columns=['iSymbol', 'ConId', "Expiry", "Strike"])
stock_chains.head()

,iSymbol,ConId,Expiry,Strike
0,ACC,44652144,20180830,860.0
1,ACC,44652144,20180830,880.0
2,ACC,44652144,20180830,900.0
3,ACC,44652144,20180830,920.0
4,ACC,44652144,20180830,940.0


In [62]:
stock_options_df = pd.merge(stock_chains, nse_options.loc[:, ['iSymbol', 'Mlot']], how='left', on='iSymbol')
stock_options_df.head()

,iSymbol,ConId,Expiry,Strike,Mlot
0,ACC,44652144,20180830,860.0,400.0
1,ACC,44652144,20180830,880.0,400.0
2,ACC,44652144,20180830,900.0,400.0
3,ACC,44652144,20180830,920.0,400.0
4,ACC,44652144,20180830,940.0,400.0


In [77]:
stock_options_df.loc[stock_options_df.isna().any(), :]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

# Reference
## A Single Options Contract

In [ ]:
# contract = Option('INFY', exchange='NSE')

contract = Option('INFY', '20180830', exchange='NSE')
contract

## Contract Details (such as Strike, Expiries and Rights)

In [ ]:
cds = ib.reqContractDetails(contract)
cds

In [ ]:
# Get symbol, strike, expiry and right
[(cd.contract.symbol, cd.contract.strike, cd.contract.lastTradeDateOrContractMonth, cd.contract.right) for cd in cds][:2]

## For Ticker Details (such as Price)

In [ ]:
options = [cd.contract for cd in cds]
options[:2]

In [ ]:
tickers = [ib.reqTickers(*options)]
tickers[:3]

### ...for a single contract's price (absolute reference)

In [ ]:
tickers[0][0].contract.strike

### ...for multiple contracts price tickers (relative reference)
(<b>Note</b>: This extracts in blocks of 10 tickers)

In [ ]:
options = [cd.contract for cd in cds]
tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]

In [ ]:
tickers[:2]

### ...for ticker information in tuples

In [ ]:
[(tickers[0][i].contract.symbol, tickers[0][i].contract.strike, 
  tickers[0][i].contract.right, tickers[0][i].bid, 
  tickers[0][i].ask, tickers[0][i].close)
 for i in range(len(tickers[0]))]

## Code to get Qualified Options List from IBKR site

In [ ]:
# Get the options list
# Note multipliers are not correct for NSE!!!

options_url1 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=1"
options_url2 = "https://www.interactivebrokers.co.in/en/index.php?f=2222&exch=nse&showcategories=OPTGRP&p=&cc=&limit=100&page=2"
nse_options = pd.concat([pd.read_html(options_url1, header=0)[2], pd.read_html(options_url2, header=0)[2]], sort=True)

# Rename the columns and add the Exchange
nse_options = nse_options.rename({'IB Symbol': 'iSymbol', 
                                  'Product Description (click link for more details)': 'Desc'}, axis=1)
nse_options['Exchange'] = 'NSE'

# List of nse indexes (extracted from error!)
idx_list = ("BANKNIFTY", "NIFTY50", "NSEFTSE", "USDINR" )

# Dataframe of equity options
nse_equity = nse_options.loc[~nse_options.iSymbol.isin(idx_list), :].reset_index(drop=True)
equity_contracts = ib.qualifyContracts(*[Stock(x, y) for x, y in zip(nse_equity.iSymbol, nse_equity.Exchange)])
equity_df = util.df(equity_contracts)

# Dataframe for index options
nse_index = nse_options.loc[nse_options.iSymbol.isin(idx_list), :].reset_index(drop=True)
index_contracts = ib.qualifyContracts(*[Index(x, y) for x, y in zip(nse_index.iSymbol, nse_index.Exchange)])
index_df = util.df(index_contracts)

In [ ]:
nse_equity.head()

In [ ]:
nse_index.head()

In [ ]:
def get_chains(scrips):
    '''Gets the option chains for the scrips passed
       Args:
          (scrips): list: qualified list of equity or index options with conId
       Returns:
          list of option chain'''
    
    # Prepare a sequence for ib.reqSecDefOptParams
    seq = [(i.symbol, '', i.secType, i.conId) for i in scrips]

    # build the chains
    chains = [ib.reqSecDefOptParams(*p) for p in seq]
    # list(map(lambda p: ib.reqSecDefOptParams(*p), seq))  # Another way of doing!
    
    return chains

index_chains = get_chains(index_contracts)